In [2]:
!wget -q https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt

In [3]:
!ls -lh

total 1.9M
-rw-r--r-- 1 root root  778 Nov  8 20:08 Assignment.ipynb
-rw-r--r-- 1 root root 1.9M Nov  8 20:08 zhivago.txt


In [298]:
import re
import string

from collections import Counter

import razdel
import nltk
import rusenttokenize

from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from tqdm.auto import tqdm

In [90]:
with open("./zhivago.txt", 'r', encoding='utf-8') as f:
    text = f.read()

## Задание 1 - Очистка

In [315]:
## удаляем xml-like теги
_text = re.sub('(\<(/?[^>]+)>)', ' ', text)
## строки из логов загрузчика
_text = re.sub('\d{2}.\d{2}.\d{4}', '', _text)
_text = re.sub('[^\S]*\.(ru)\S*', '', _text)
_text = re.sub('\d{1}.\d{1}', '', _text)
## Цифры, Латиницу(логи загрузчика), скобочки(ибо зачем)
_text = re.sub('[0-9a-zA-Z«»]', '', _text)
## Странные штуки в конце
_text = re.sub("[/+]", '', _text)
## -
_text = re.sub('\s–\s', '', _text)
## лишние пробелы
text = re.sub("\s+", ' ', _text)

## Задание 2 - токенизация/разделение

In [316]:
punctuation = "".join((set(string.punctuation) - set(".")))

In [317]:
text = text.translate(str.maketrans('', '', punctuation)).strip()

In [318]:
sentences = rusenttokenize.ru_sent_tokenize(text)

In [320]:
## Приводим к нижнему регистру после токенизации, т.к отсутствие регистра может повлиять на корректность токенизации
tokenized_sentences = [
    tuple(token.text.lower() for token in razdel.tokenize(sentence)) for sentence in tqdm(sentences)
]

### 2.1 - Повторяющиеся предложения

In [321]:
counter = Counter(tokenized_sentences)

In [322]:
repeating_sentences = list(map(
    lambda x: (" ".join(x[0]), x[1]), # детокенизируем для отображения
    filter(
        lambda x: x[1] >= 2 and x[0][0] != '–', # встречающиеся два и более раз, не являющиеся прямой речью (начинается с -)
        counter.most_common()
    )
))

Повторяющиеся предложения есть, всего их(без учета прямой речи) -

In [323]:
print(len(repeating_sentences))

58


Примеры таких предложений:

In [324]:
repeating_sentences[:10]

[('да .', 10),
 ('сеялки .', 4),
 ('но дело не в этом .', 3),
 ('молотилки .', 3),
 ('свеча горела на столе свеча горела .', 3),
 ('единственно живое и яркое в васэто то что вы жили в одно время со мной и меня знали .',
  2),
 ('весной в несколько дней лес преображается подымается до облаков в его покрытых листьями дебрях можно затеряться спрятаться .',
  2),
 ('это превращение достигается движением по стремительности превосходящим движения животных потому что животное не растет так быстро как растение и которого никогда нельзя подсмотреть .',
  2),
 ('лес не передвигается мы не можем его накрыть подстеречь за переменою места .',
  2),
 ('мы всегда застаем его в неподвижности .', 2)]

### 2.2 - Самый частотный токен

In [325]:
frequencies = Counter()
for sentence in tokenized_sentences:
    frequencies.update(sentence)

In [326]:
most_frequent = list(filter(lambda x: len(x[0]) > 6, frequencies.most_common()))[0]

In [327]:
print(f"Самый частотный токен длинее 6 символов - {most_frequent[0]}, он встречается {most_frequent[1]} раз")

Самый частотный токен длинее 6 символов - андреевич, он встречается 285 раз


## 3 - Стемминг

In [328]:
stemmer = SnowballStemmer('russian')

In [329]:
all_words = list([word for sentence in tokenized_sentences for word in sentence])

In [330]:
stemmed_words = list(map(stemmer.stem, all_words))

### 3.2 Слово не изменилось после стеммизации
Если интерпретировать такую ошибку как сказано в условии - то таких ошибок очень много

In [334]:
## ошибки не-стеммизации
non_stemmed_idx = list(filter(lambda x: len(x[1]) > 4 and x[1] == stemmed_words[x[0]], enumerate(tqdm(all_words))))

In [335]:
len(non_stemmed_idx)

6661

Однако я думаю что большинство из них не представляют собой действительно ошибки, просто слово является само себе "стеммой"

In [336]:
for i, w in non_stemmed_idx[:30]:
    print(f"{i:^5}|{all_words[i]:^20} == {stemmed_words[i]:^20}")

  0  |       борис         ==        борис        
  1  |     леонидович      ==      леонидович     
  2  |     пастернак       ==      пастернак      
  3  |       доктор        ==        доктор       
  5  |       доктор        ==        доктор       
 18  |       принес        ==        принес       
 45  |       доктор        ==        доктор       
 63  |      человек        ==       человек       
 66  |       пишет         ==        пишет        
 107 |       жертв         ==        жертв        
 114 |       перед         ==        перед        
 136 |       строк         ==        строк        
 140 |       могут         ==        могут        
 162 |       борис         ==        борис        
 163 |     пастернак       ==      пастернак      
 164 |       доктор        ==        доктор       
 167 |       дышат         ==        дышат        
 177 |       доктор        ==        доктор       
 179 |       борис         ==        борис        
 180 |     пастернак       ==  

Но и ошибки тоже есть, см. слова #260 и #269 - две словоформы одной лексемы ("будет"), и они не изменились после стеммирования, хотя нужно было бы

### 3.1 Одна стемма для разных слов

In [337]:
stem2words = {}
for i, word in enumerate(all_words):
    stemm = stemmed_words[i]
    if stemm not in stem2words:
        stem2words[stemm] = set()
        
    stem2words[stemm].add(word)

Будем смотреть такие слова, длина формы которых очень сильно отличается от длины стеммы

In [338]:
import numpy as np
error_pairs = {}
for key, forms in stem2words.items():
    if len(key) <= 6 and np.mean([abs(len(key) - len(form)) for form in forms]) >= 5 and len(forms) > 2:
        error_pairs[key] = forms

In [339]:
list(error_pairs.items())

[('оста',
  {'оставшегося',
   'оставшееся',
   'оставшейся',
   'оставшеюся',
   'оставшиеся',
   'оставшимися',
   'оставшимся',
   'оставшись',
   'оставшихся',
   'оставшуюся',
   'остаемся',
   'остается',
   'осталась',
   'остались',
   'осталось',
   'остался',
   'остаться',
   'остаются',
   'остающееся',
   'остающемся',
   'остающимися',
   'остающимся'}),
 ('сохран',
  {'сохрани',
   'сохранив',
   'сохранившая',
   'сохранившаяся',
   'сохранившегося',
   'сохранившейся',
   'сохранившиеся',
   'сохранившимся',
   'сохранившихся',
   'сохранились',
   'сохранилось',
   'сохранился',
   'сохранить',
   'сохранности',
   'сохранны'}),
 ('выс', {'выси', 'высившаяся', 'высившейся', 'высившуюся', 'высилась'}),
 ('появ',
  {'появившаяся',
   'появилась',
   'появились',
   'появилось',
   'появился',
   'появиться'}),
 ('прокат', {'прокатившегося', 'прокатились', 'прокатился', 'прокатиться'}),
 ('заблуд',
  {'заблудившаяся',
   'заблудившиеся',
   'заблудившийся',
   'заблудилс

Среди таких слов можно найти несколько примеров ошибок, удовлетворяющих условию
- 'пузыр': пузырившегося(деепричастие?) и пузырями(сущ.)
- 'выси': выси(сущ.) и высившаяся(деепричастие?)

## 4 - список стоп-слов из nltk

In [340]:
stop_words = stopwords.words('russian')

Посмотрим на самые частотные слова в нашем тексте, которые не встречаются в stopwords и посмотрим какие из них можно туда добавить

In [341]:
freq_words = {k:v for k, v in frequencies.most_common(200)}

In [342]:
dissset = set(freq_words.keys()) - set(stop_words)
print(dissset)

{'это', 'время', 'поезд', 'тебе', 'андреевич', 'люди', 'доктор', 'жизни', 'своим', 'доме', 'которых', 'наверное', 'глаза', 'дома', 'точно', 'которой', 'вместе', 'стали', 'кроме', 'дом', 'сама', 'доктора', 'эта', 'всем', 'человек', 'часть', 'правда', 'нам', 'знаю', '...', 'словно', 'друг', 'ночь', 'очень', 'юрия', 'юрий', 'чтото', '.', 'день', 'нем', 'живаго', 'стало', 'окна', 'говорит', 'времени', 'кругом', 'оно', 'этим', 'лара', 'свете', 'который', 'андреевича', 'этих', 'юра', 'руки', 'которые', 'пока', 'своей', 'стал', 'ними', 'несколько', 'голову', 'дело', 'своих', 'жизнь', 'минуту', 'конца', 'сторону', 'весь'}


Первые четыре слов, которые можно добавить в стоп-слова - это вариации слова 'это': 
- это
- эта
- этим
- этих

Почему:
1. В стоп словах уже есть несколько вариаций слова 'это': этот, этого, этом, эти, эту, этой. Поэтому, следуя той же логике, можно добавить отсутствующие вариации, которые мы видим в нашем тексте

In [343]:
list(filter(lambda x: x.startswith('э'), stop_words))

['этот', 'этого', 'этом', 'эти', 'эту', 'этой']

2. Они очень часто встречаются в тексте, например 'это' встречается 1001 раз

In [344]:
freq_words['это']

949

Пятое слово - это 'оно'

Почему его стоит добавить в список стоп-слов: в списке уже есть аналогичные слова для м.р, ж.р, и мн. числа. Выглядит как ошибка, что в списке нет формы для среднего рода.

In [345]:
list(filter(lambda x: x.startswith('он'), stop_words))

['он', 'она', 'они']

## Задание 5 - лемматизация

In [392]:
mystem = Mystem()
pymorhy = MorphAnalyzer()
vocab = list(set(all_words))

In [393]:
mystem_lemms = list(map(lambda x: mystem.lemmatize(x)[0], tqdm(vocab)))

In [394]:
pymorphy_lemms = list(map(lambda x: pymorhy.normal_forms(x)[0], tqdm(vocab)))

In [395]:
mismatch = [
    (frequencies[vocab[i]], vocab[i], mystem_lemms[i], pymorphy_lemms[i]) 
    for i in range(len(vocab)) 
    if (mystem_lemms[i] != pymorphy_lemms[i])
]

In [396]:
mismatch = sorted(mismatch, key=lambda x: x[0], reverse=True)

In [399]:
print("|# occurs|    word     |   mystem3    |   pymorphy2   |")
print("-------------------------------------------------------")
for count, vocab, lemma_mystem, lemma_pymorphy in mismatch[:50]:
    print(f"|{count:^8}|{vocab:^13}|{lemma_mystem:^14}|{lemma_pymorphy:^15}|")

|# occurs|    word     |   mystem3    |   pymorphy2   |
-------------------------------------------------------
|  753   |     все     |     все      |      всё      |
|  349   |     еще     |     еще      |      ещё      |
|  234   |     со      |      со      |       с       |
|  206   |     во      |      во      |       в       |
|  202   |     чем     |     что      |      чем      |
|  161   |     ним     |      он      |      они      |
|  161   |    может    |    может     |     мочь      |
|  141   |   больше    |    больше    |    большой    |
|  132   |    того     |      то      |      тот      |
|  122   |    чтото    |    чтото     |     чтоть     |
|  114   |    есть     |     быть     |     есть      |
|  114   |    всех     |     все      |     весь      |
|  108   |     тем     |      то      |      тем      |
|  103   |    стал     | становиться  |     стать     |
|   97   |    всем     |     все      |     весь      |
|   94   |    дома     |     дома     |      дом

Из результатов анализа мисматчей для наиболее частотных слов, видно, что каждая библиотека имеет свои проблемы:
1. Pymorphy2 подвержен Gender Bias: имена/отчества/фамилии в ж.р нормализуются в аналогичные, но в м.р. 

Mystem так не ошибается, скорее всего из-за того что у него словарь меньшего размера и он обрабатывает такие слова как ошибки

2. Mystem некорректно лемматизирует некоторые устаревшие формы предлогов, см. 'ко', 'об', 'со', 'во'.
3. Pymorphy2 исправляет ошибки в ходе лемматизации слов, которые должны быть написаны через 'ё', но написаны с 'е', см. вперед, еще, все(вероятно)
4. Иногда это исправление некорректно, см. дальше. В т.ч исправления ошибок другого типа тоже могут быть некорректны, см. ктото, изза, гдето, какойто, чтото. В тоже время Mystem не делает ничего с словами, написанными с ошибкой.

В итоге можно сказать, что для анализа текстов такого рода больше подходит mystem.